# Assignment 6

### Part A

In [1]:
import numpy as np
import pandas as pd

Consider a DataFrame giving information about the price of a stock in 1 year, and the probability of that price occurring.

In [2]:
data = pd.DataFrame(data = {"Price in 1 Year" : [32, 43, 54],
                            "Probability" : [0.3, 0.4, 0.3]})
                    
data

Price in 1 Year  Probability
0               32          0.3
1               43          0.4
2               54          0.3

Write the function ```exp_return```, which calculates the expected return of the stock, given the current stock price ```P```.

In [3]:
def exp_return(data, P):
    # Write your code here
    sum = 0
    for i in range(len(data)):
        sum += data.loc[i,'Probability'] * (data.loc[i, 'Price in 1 Year'] / P - 1)
    # exp_return(data, 40) should output 0.07500000000000002
    return sum

Write the function ```volatility```, which calculates the expected return of the stock, given the current stock price ```P```.

In [5]:
def volatility(data, P):
    # Write your code here
    sum = 0
    for i in range(len(data)):
        sum += data.loc[i,'Probability'] * ((data.loc[i, 'Price in 1 Year'] / P - 1)**2)
    rate = exp_return(data, P)
    sum -= rate**2
    volatility = np.sqrt(sum)
    # volatility(data,40) should output 0.21301408404140795
    return volatility

Write the function ```interval```, which calculates the 95% confidence interval for the expected return of the data. The corresponding Z score is 1.96. The function should return a list of size 2, with the first element representing the lower bound, and the second element representing the upper bound. 

In [7]:
def interval(data, P):
    # Write your code here
    # interval(data, 40) should output [-0.16604812797447732, 0.3160481279744774]
    
    # Calculate the expected value (mean)
    expected_value = exp_return(data, P)
    
    # Calculate the standard error of the mean
    standard_error = volatility(data, P)
    
    # Calculate the margin of error
    margin_of_error = 1.96 * (standard_error / np.sqrt(len(data)))
    
    # Calculate the confidence interval
    lower_bound = expected_value - margin_of_error
    upper_bound = expected_value + margin_of_error
    
    return [lower_bound, upper_bound]

Write the function ```exp_return_capm```, which, given the beta of a security ```beta```, the return on the market ```market_return```,  and the risk free interest rate ```rf```, calculates the expected return of the security.

In [9]:
def exp_return_capm(beta, market_return, rf):
    # Write your code here
    exp_return = rf + beta * (market_return-rf)
    # exp_return_capm(-0.6,0.1, 0.02) should output -0.028
    return exp_return

### Part B

Write the function ```clean_data```, which will take a path to a csv file like the one provided. This function should remove all observations with a non number ```RET```, and remove all securities with (strictly) less than 500 observations. Your returned DataFrame should have 4 (non-index) columns, `PERMNO`, `date`, `CUSIP`, and `RET`.

In [11]:
def clean_data(path):
    # Write your code here
    # This should yield data with 403316 rows × 4 columns
    
    
    # Read the CSV file into a DataFrame
    df = pd.read_csv(path)

    # Filter out observations with a non-number RET
    df = df[pd.to_numeric(df['RET'], errors='coerce').notna()]

    # Group by PERMNO and count the number of observations
    counts = df.groupby('PERMNO').size()

    # Filter out securities with less than 500 observations
    valid_permnos = counts[counts >= 500].index
    df = df[df['PERMNO'].isin(valid_permnos)]

    # Keep only the desired columns
    df = df[['PERMNO', 'date', 'CUSIP', 'RET']]

    return df

Write the function ```avg_return```, which will calculate the average return for each security. This function should take in a DataFrame ```data```, formatted like one outputted by the function ```clean_data```. The results should be returned in a DataFrame formatted like the one supplied.

Additionally, write the function ```security_volatilities```, which does the same thing as ```avg_return```, except it returns the volatilities of the returns of the securities.

In [12]:
def avg_return(data):
    # Write your code here
    # This should yield data with 679 rows × 2 columns
    
    data['RET'] = pd.to_numeric(data['RET'])
    
    output = pd.DataFrame(data = {
         "Average Return" : data.groupby('CUSIP')['RET'].mean()
    }).reset_index()
    
    return output

In [13]:
def security_volatilities(data):
    # Write your code here
    # Thids should yield data with 679 rows × 2 columns
    
    data['RET'] = pd.to_numeric(data['RET'])
    
    output = pd.DataFrame(data = {
         "Volatility" : data.groupby('CUSIP')['RET'].std()
    }).reset_index()
    
    return output

Write the function ```correlation```, which will return the correlation between average return and volatility. This function takes two DataFrames, ```ret_data``` and ```vol_data```, which are the outputs of ```avg_return``` and ```security_volatilities``` respectively.

In [14]:
def correlation(ret_data, vol_data):
    # Write your code here
    # correlation(rets, vols) should yield 0.44837090727059614
    
    combined_data = ret_data.merge(vol_data, on='CUSIP', how='inner')
    
    # Calculate the correlation between 'Average Return' and 'Volatility'
    correlation_value = combined_data['Average Return'].corr(combined_data['Volatility'])
    
    return correlation_value